In [40]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import os
import matplotlib.pyplot as plt
import base64
from konlpy.tag import Okt
import re
from collections import Counter
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')


In [41]:
okt = Okt()

## content에서 키워드 추출
def preprocess_text(text):
    # 알파벳이 아닌 문자 정규화 및 제거
    text = re.sub("[^가-힣a-zA-Z ]", "", text)
    # 토큰화, 명사추출
    tokens = okt.nouns(text)  # 명사 추출
    tokens = [n for n in tokens if len(n) > 1]  # 길이가 2이상인 단어만 추출
    return tokens


In [42]:
def pearson_correlation_and_p_value_matrix(df):
    # Number of merchants
    n = df.shape[0]
    # Initialize the correlation matrix and the p-value matrix
    correlation_matrix = np.zeros((n, n))
    p_value_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            if i != j:
                data_i = df.iloc[i, 1:].values
                data_j = df.iloc[j, 1:].values
                correlation, p_value = pearsonr(data_i, data_j)
                correlation_matrix[i, j] = correlation
                p_value_matrix[i, j] = p_value
            else:
                correlation_matrix[i, j] = 1  # 자기 자신 상관계수는 1
                p_value_matrix[i, j] = 0  # 자기 자신 p-value는 0
    
    return correlation_matrix, p_value_matrix

#### 내가 원하는 음식점 넣어도 198개 중에 추천하게


In [43]:
def find_similar_restaurants(restaurant_name, top_n = 10, weight_similarity = 0.7, weight_stars = 0.3):
    # 데이터 로드
    weights_df = pd.read_csv('./알고리즘 데이터/weights.csv')
    cor_matrix = np.load('./알고리즘 데이터/cor.npy')
    total_rating = pd.read_csv('../data2/total_rating.csv')

    # 음식점 이름과 별점 목록 가져오기
    restaurant_names = weights_df['merchant'].values
    stars = total_rating['total_rating'].values  # 별점 데이터

    ## 검색 음식점이 음식점 리스트에 있는 경우
    if restaurant_name in restaurant_names:
        # 주어진 음식점의 인덱스 찾기 
        idx = np.where(restaurant_names == restaurant_name)[0][0]
        
        # 유사도 점수와 인덱스 가져오기
        similarity_scores = cor_matrix[idx]

        # 자기 자신을 결과에서 제외
        similarity_scores[idx] = -np.inf
        
        # 종합 점수 계산: 유사도 점수와 별점을 가중치로 조정
        normalized_stars = (stars / stars.max())  # 별점 정규화
        total_scores = weight_similarity * similarity_scores + weight_stars * normalized_stars
        
        # 종합 점수에 따라 인덱스 정렬 및 상위 N개 선택
        sorted_indices = np.argsort(total_scores)[::-1][:top_n]
        
         # 결과 출력
        print(f"Top {top_n} restaurants similar to {restaurant_name} based on similarity and stars:")
        for i in sorted_indices:
            print(f"{restaurant_names[i]} - Total Score: {total_scores[i]}, Similarity: {similarity_scores[i]}, Stars: {stars[i]}")
    

    ## 검색 음식점이 음식점 리스트에 없는 경우
    else:
        
        ##################################################### 데이터 가져오는 코드

        # 카카오 맵
        driver = webdriver.Chrome()
        driver.get('https://map.kakao.com/')
        time.sleep(1)

        ## input 창에 검색어 입력
        searchbox = driver.find_element(By.ID, 'search.keyword.query')
        searchbox.send_keys(restaurant_name+ Keys.ENTER)
        time.sleep(2)
    
        # 현재 리스트의 상세보기 클릭
        driver.find_element(By.CSS_SELECTOR, 'a[data-id="moreview"]').click()

        time.sleep(1)

      

        # driver에서 탭을 이동
        driver.switch_to.window(driver.window_handles[1])

        time.sleep(0.2)

         # 영업시간 더 보기 클릭
        try:
            oper_more = driver.find_elements(By.CSS_SELECTOR, 'a[data-logevent = "main_info,more_timeinfo"]')
            oper_more[0].click()
            time.sleep(2)  

        # 영업시간 더보기가 없는 경우
        except:
            pass

        # 후기 더보기 클릭
        try:
            link_more = driver.find_element(By.CLASS_NAME, 'txt_more')
            while link_more.text == "후기 더보기":
                link_more = driver.find_element(By.CLASS_NAME, 'txt_more')
                # 마지막 후기 더보기까지 눌렀으면 멈추기
                if link_more.text != "후기 접기":
                    link_more.click()
                    time.sleep(0.3)
        # 후기 더보기 없는 경우        
        except:
            pass
        time.sleep(1)
  
        ## 상세page bs4 파싱 분석
        soup = bs(driver.page_source, 'html.parser')

        driver.close()
        # driver에서 탭을 이동
        driver.switch_to.window(driver.window_handles[0])

        driver.close()
        
        ## 정보 넣을 빈 딕셔너리
        data = {
            'merchant' : [],
            'category' : [],
            'star' : [],
            'address' : [],
            'oper_time' : [],
            'starCount' : [],
            'reviewCount' : []
            }
        data2 = {
            'merchant' : [],
            'user_name' : [],
            'user_rank' : [],
            'num_response' : [],
            'user_star' : [], 
            'time' : [],
            'rating' : [],
            'content' : []
            }
        
        
        ## 매장명
        mer = soup.find('h2', {'class':'tit_location'}).get_text()
        data['merchant'].append(mer)
        ## 분류
        data['category'].append(soup.find('span',{'class':'txt_location'}).get_text().split(':')[1].strip())
        ## 주소
        data['address'].append(soup.find('span', {'class': 'txt_address'}).get_text().replace(" ", "").replace("\n", " "))

        ## 별점, 리뷰 수 
        # 별점 있는 경우
        if len(soup.find_all('span',{'class':'color_b'})) > 1:
            data['star'].append(soup.find_all('span', {'class':'color_b'})[0].get_text().strip('점'))
            data['starCount'].append(soup.find('a', {'class': 'link_evaluation'})['data-cnt'])
            data['reviewCount'].append(soup.find_all('span', {'class':'color_b'})[1].get_text().strip('개'))
        # 별점 없는 경우
        else:
            data['star'].append('0')
            data['starCount'].append('0')
            data['reviewCount'].append(soup.find('span', {'class':'color_b'}).get_text().strip('개'))

        ## 영업시간
        # 더보기 있는 경우
        if oper_more:    
            data['oper_time'].append(soup.find_all('ul', {'class':'list_operation'})[1].get_text().strip())
        # 더보기 없는 경우
        else:
            data['oper_time'].append(soup.find('ul', {'class':'list_operation'}).get_text().strip().replace("\n", ""))


        # 리뷰 목록 잡기
        reveiw_listed = soup.find_all('li', {'data-ismy':'false'})

        #### df2 만들기 (리뷰df)
        data2['merchant'] = mer
        for j in range(len(reveiw_listed)):
            ## 유저 이름
            data2['user_name'].append(soup.find_all('span',{'class':'txt_username'})[j].get_text())
            ## 유저 레벨
            data2['user_rank'].append(soup.find_all('span', {'class' : 'txt_badge'})[j].get_text().strip('레벨'))
            ## 유저 후기수
            data2['num_response'].append(
                soup.find_all('div',{'class':'unit_info'})[j].find_all('span',{'class':'txt_desc'})[0].get_text()
            )
            ## 유저 별점 평균
            data2['user_star'].append(
                soup.find_all('div',{'class':'unit_info'})[j].find_all('span',{'class':'txt_desc'})[1].get_text()
            )
            ### 리뷰 작성 시간
            data2['time'].append(
                soup.find_all('div',{'class':'unit_info'})[j].find('span',{'class':'time_write'}).get_text()
            )
            ## 리뷰 내용
            data2['content'].append(
                soup.find_all('p', {'class':'txt_comment'})[j].get_text().strip('더보기')
            )

            ## 유저가 평가한 별점 
            # 너비를 추출하여 별점 계산 (너비는 별점의 백분율을 나타냄)
            style_attribute = soup.find_all('div', {'class':'star_info'})[j].find('span',{'class':'ico_star inner_star'})['style']
            width_percent = float(style_attribute.split(':')[1].strip('%;'))
            rating_out_of_five = width_percent / 20  # 별점은 100%를 5로 나눈 값
            data2['rating'].append(rating_out_of_five)

        ## 데이터 프레임으로 만들기
        df2 = pd.DataFrame(data2)
        # 리뷰내용 없는 것 결측치로 대체
        df2.replace('',np.nan, inplace= True)
        

        df = pd.DataFrame(data)
        # test중이라 일부러 데이터 저장하지 않음.
        
        print(f"{mer}데이터 저장완료")

        ## 전처리
        # content 결측치 행 제거
        df2_clean = df2.dropna(subset=['content'], ignore_index=True)
        df2_clean

        # 추출한 단어(keywords) 열 생성
        df2_clean['keywords'] = df2_clean['content'].apply(preprocess_text)
        df2_clean


        a = '아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓'
        a = a.replace(" ", ",")

        ## 불용어 제거

        

        # 불용어 정의
        stopwords = [a,'맛있어요', '맛있고', '좋아요', '맛', '맛은', '맛이', '좋고', '안', '엄청', '맛있게', '맛도', '다른', '먹고', 
            '분위기', '맛있음', '맛있었어요', '같아요', '여기', '그리고','여기', '맛', '너무', '진짜', '다', '좀', '정말', '잘', '더', '그냥', '또', '것', '수', '많이', '꼭', '이', '거', '것은', '에서', '에는', 
            '있습니다', '있어', '있는', '입니다', '하는', '하기', '하고', '했어요', '했습니다', '한', '할', '합니다', '해', '해요', 
            '으로', '로는', '와', '과', '도', '고', '에', '에게', '와서', '와의', '으로부터', '이다', '있다', '되다', '같다']

        # 불용어 제거
        df2_clean['filtered_words'] = df2_clean['keywords'].apply(lambda words: [word for word in words if word not in stopwords])

        # 빈도수 계산
        df2_filtered_words = [word for sublist in df2_clean['filtered_words'] for word in sublist]
        df2_filtered_word_counts = Counter(df2_filtered_words)

        ## 가중치 계산

        # 유의어 그룹 정의
        synonym_groups = {
            "western": ["양식","피자","파스타","버거","샐러드","스테이크","화덕","시카고피자","레스토랑","피클"],
            "japan": ["일식","라멘","우동","튀김","초밥","츠케멘","소바","차슈","카츠","회"],
            "service": ["서비스","친절","퀄리티","위생","설명","상태","관리","청결","정성","합리"],
            "location": ["스타일","컨셉","인스타","특색","감성","뷰","테라스","테마","야외","예술"],
            "meat": ["고기","곱창","돼지","닭갈비","등심","갈비","대창","삼겹살","육즙","양고기"],
            "alchol": ["술","국물","안주","맥주","술집","막걸리","소주","한잔","생맥주", "칵테일"],
            "cafe": ["카페","딸기","케이크","음료","빵","디저트","라떼","빙수","케익","생크림"],
            "time": ["시간","근처","거리","동네","예약","대기","웨이팅","주변","멀리","가까"]
        }

        # 각 리뷰의 단어 목록에서 그룹 단어의 빈도를 계산하는 함수 (횟수에 따라 0,1,2 식으로 출력)
        def calculate_group_weights(words, group_words):
            return sum(word in group_words for word in words)

        # 각 그룹에 위 함수 적용
        for group_name, group_words in synonym_groups.items():
            df2_clean[group_name] = df2_clean['filtered_words'].apply(lambda words: calculate_group_weights(words, group_words))

        # 음식점별로 빈도를 다 더한다.
        grouped_weights = df2_clean.groupby('merchant')[list(synonym_groups.keys())].sum()

        # 가중치 계산
        normalized_weights = grouped_weights.div(df2_clean.groupby('merchant').size(), axis=0)

        weights_df2 = normalized_weights.reset_index()

        weights_df = pd.read_csv('./알고리즘 데이터/weights.csv')

        weights_df = pd.concat([weights_df, weights_df2], ignore_index=True)



        ## 피어슨 계산
        cor_matrix, p_value_matrix= pearson_correlation_and_p_value_matrix(weights_df)

        
#########################################################################################################        
        
        total_rating = pd.read_csv('../data2/total_rating.csv')

        restaurant_names = weights_df['merchant'].values
        stars = total_rating['total_rating'].values  # 별점 데이터

        # 주어진 음식점의 인덱스 찾기
        idx = np.where(restaurant_names == restaurant_name)[0][0]
        
        # 유사도 점수와 인덱스 가져오기
        similarity_scores = cor_matrix[idx]

        # 자기 자신을 결과에서 제외
        similarity_scores = similarity_scores[:-1]
        
        # 종합 점수 계산: 유사도 점수와 별점을 가중치로 조정
        normalized_stars = (stars / stars.max())  # 별점 정규화
        total_scores = weight_similarity * similarity_scores + weight_stars * normalized_stars
        
        # 종합 점수에 따라 인덱스 정렬 및 상위 N개 선택
        sorted_indices = np.argsort(total_scores)[::-1][:top_n]
        
        # 결과 출력
        print(f"Top {top_n} restaurants similar to {restaurant_name} based on similarity and stars:")
        for i in sorted_indices:
            print(f"{restaurant_names[i]} - Total Score: {total_scores[i]}, Similarity: {similarity_scores[i]}, Stars: {stars[i]}")
    

In [48]:
find_similar_restaurants('서울초밥 당산점')

서울초밥 당산점데이터 저장완료


C:\Users\Jws\AppData\Local\Temp\ipykernel_2520\736546709.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_clean['keywords'] = df2_clean['content'].apply(preprocess_text)
C:\Users\Jws\AppData\Local\Temp\ipykernel_2520\736546709.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_clean['filtered_words'] = df2_clean['keywords'].apply(lambda words: [word for word in words if word not in stopwords])
C:\Users\Jws\AppData\Local\Temp\ipykernel_2520\736546709.py:241: SettingWithCopyWarning: 
A value i

Top 10 restaurants similar to 서울초밥 당산점 based on similarity and stars:
여우골 홍대서교점 - Total Score: 0.8491676701440277, Similarity: 0.9493531940979573, Stars: 0.7084507042253522
가미우동 - Total Score: 0.7563697023497499, Similarity: 0.9204237434000185, Stars: 0.4300621118012423
오레노라멘 합정본점 - Total Score: 0.7517463242251488, Similarity: 0.8515271740583994, Stars: 0.5973861720067454
칸다소바 홍대점 - Total Score: 0.7425249268920838, Similarity: 0.9478420693758274, Stars: 0.3032857142857143
소코아 홍대점 - Total Score: 0.7319860768332085, Similarity: 0.9419319436631992, Stars: 0.2787199999999999
여우골 홍대점 - Total Score: 0.7264825772926793, Similarity: 0.9570414928824209, Stars: 0.2170149253731343
츠케루 - Total Score: 0.7226964907488441, Similarity: 0.9389420860560254, Stars: 0.2511038961038961
지로우라멘 - Total Score: 0.6840720463433576, Similarity: 0.8987764609610687, Stars: 0.2107792207792208
또보겠지떡볶이집 호호피크닉점 - Total Score: 0.6812622135576853, Similarity: 0.9756935987955934, Stars: -0.0066129032258064
토모토카레 홍대점 - Tot